<a href="https://colab.research.google.com/github/lamcnguyen89/2D_Image_to_3D_Model/blob/main/Assignment_04/Assignment_04_Mistral_and_Mixtral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 4: Mistral vs Mixtral Pipeline**



*   Got to https://people.eecs.berkeley.edu/~hendrycks/data.tar
*   Colab should have both mixtral and mistral 7b pipelines
*   Compare both in terms of speed and accuracy
*   Bonus: 100 points for applying SMoE to other models
*   Due 03Sep2024





**Mistral-7b** \
Mistral-7B is one of the biggest and most advanced Large Language Models out there, trained on a massive dataset of text and code.

Link: https://huggingface.co/mistralai/Mistral-7B-v0.1/discussions/13


**Mixtral 8x7b** \
Mixtral 8x7b is a high-quality sparse mixture of experts (SMoE) model with open weights created by Mistral AI. It outperforms Llama 2 70B on most benchmarks and batches or beats GPT3.5 on most standard benchmarks.

Link: https://github.com/dvmazur/mixtral-offloading/blob/master/notebooks/demo.ipynb


# **Load Mistral-7B Model and Test**

In [1]:
# Download Mistral-7B Model
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip install datasets
!pip install --upgrade huggingface_hub

# You need to go onto Google Colab and create a variable in Secrets. This variable will contain a token for the huggingface account that we will get our pretrained models from
from google.colab import userdata
secret = userdata.get('HF_TOKEN')
!huggingface-cli login --token {secret}




  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 31.2 MB/s et

NameError: name 'clear_output' is not defined

In [ ]:
!nvidia-smi

Tue Sep  3 02:07:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments

torch.set_default_device('cuda')



In [ ]:
# Import Pretrained model and load into memory
# Note I required a GPU with 40gb of memory for this project.
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             torch_dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                          torch_dtype="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# How to Prompt Mistral AI Models: https://community.aws/content/2dFNOnLVQRhyrOrMsloofnW0ckZ/how-to-prompt-mistral-ai-models-and-why


text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Oh, absolutely! Here's a classic homemade mayonnaise recipe that's sure to impress:

Ingredients:

* 2 large egg yolks
* 1 tablespoon dijon mustard
* 2 tablespoons white wine vinegar
* 1 clove garlic, minced
* 1/2 teaspoon salt
* 1/2 cup olive oil
* 1/4 cup vegetable oil
* Fresh herbs or spices, to taste (optional)

Instructions:

1. In a small bowl, whisk together the egg yolks, dijon mustard, white wine vinegar, minced garlic, and salt until well combined.
2. Slowly stream in the olive oil and vegetable oil, whisking continuously to create a smooth, creamy sauce.
3. Taste and adjust seasoning as desired.
4. Store any leftover mayonnaise in an airtight container in the fridge for up to 3 days.

And there you have it! A simple, yet de

# **Load and Process Dataset**


In [ ]:
# Source: https://github.com/mddunlap924/PyTorch-LLM/blob/main/notebooks/training.ipynb
!ls
validation_dataset = load_dataset("Assignment_04_Dataset")
print(validation_dataset)

Assignment_04_Dataset  sample_data
DatasetDict({
    validation: Dataset({
        features: ['Question', 'Answer_01', 'Answer_02', 'Answer_03', 'Answer_04', 'Answer_05'],
        num_rows: 1516
    })
})


# **Load Mixtral Pipeline**

In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate xformers einops

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

device='cuda'

model_id = "mistralai/Mixtral-8x7B-v0.1"
bnb_config = transformers.BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)

model = transformers.AutoModelForCausalLM.from_pretrained(
model_id,
trust_remote_code=True,
quantization_config=bnb_config,
).to(device)

tokenizer = transformers.AutoTokenizer.from_pretrained(
model_id,
)

text = "What is the meaning of life?"
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)
model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


What is the meaning of life? Is it to get married and have kids? Is there a reason for that either?

We are living in the best possible times. If we're bored, surely something is wrong.
User 21: > Do what you want? Why? What if you have family who'll be left alone?

If you have responsibilities to your family, it's a little different. If they can't live without you, that's a problem.

>If there is no meaning of life, why should we live for ourselves either?

Because it's pleasurable.
User 17: You just defined the meaning of life. It is to feel pleasure and avoid pain.
User 21: Not really, the meaning of life is whatever someone feels it is.
User 17: Well, then you have to define a 'thing'. Does a tree have a meaning to its life? Do rocks and
